In [2]:
import numpy as np
import pandas as pd
import sys
from sklearn import preprocessing
from keras.utils.np_utils import to_categorical

MAX_SEQUENCE_NUM = 777

def mapPhones():
    phone2num = dict()
    with open('./48phone_char.map') as f:
        lines = f.readlines()
        for line in lines:
            line = line.rstrip('\n')           
            phones = line.split('\t')
            phone2num[phones[0]] = phones[1]
    phone39 = dict()
    with open('./48_39.map') as f:
        
        lines = f.readlines()
        for line in lines:
            line = line.rstrip('\n')
            phones = line.split('\t')
            phone39[phones[0]] = phones[1]
    
    labelList = []
    for key, value in phone39.items():
        label = phone2num[value]
        if(label not in labelList):
            labelList.append(label)
   
    
    return phone2num, phone39, labelList


def loadData():
    phone2num, phone39, labelList  = mapPhones()
    print(labelList)

    # fbank_features = ['fb_' + str(i) for i in range(0,69)]
    mfcc_features = ['mfcc_' + str(i) for i in range(0,39)]
    # fbank_train = pd.read_csv('./fbank/train.ark', sep=' ', header = None, index_col=0, names = ['id'] + fbank_features)
    mfcc_train = pd.read_csv('./mfcc/train.ark', sep=' ', header = None, index_col=0, names = ['id'] + mfcc_features)

    labels = pd.read_csv('train.lab', sep=',', header = None, index_col=0, names = ['id', 'label'])
    

    df = pd.concat([mfcc_train, labels], axis=1)
    # map label frome phone48 -> phone39 -> number -> index of number in labelList -> +1
    for key, value in phone39.items():        
        df.loc[df['label'] == key, 'label'] = labelList.index(phone2num[value]) + 1
    df['f_id'] = df.index
    df['fid'] = df['f_id'].apply(lambda x: x.split('_')[2])
    df[['fid']]= df[['fid']].apply(pd.to_numeric)
    df[['label']]= df[['label']].apply(pd.to_numeric)
    df['f_name'] =  df['f_id'].apply(lambda x: x.split('_')[0] + '_' + x.split('_')[1])
    del df['f_id']

    

    df = df.sort_values(by=['f_name', 'fid'])
    df_g = df.groupby('f_name')
    df_g = np.array(list(df_g))
    
    df_g = np.delete(df_g, 0, 1)
    X_data = []
    y_data = []

    for rows in (df_g):
        labels =  rows[0].as_matrix(['label'])
        
        labels = to_categorical(labels, num_classes = 40)
      
        mfcc = rows[0].as_matrix(mfcc_features)
       
       
        
        
        mfcc = preprocessing.scale(mfcc)
    #         row = (row - np.mean(row, axis = 1)) / np.std(row, axis = 1)
        padding_num = MAX_SEQUENCE_NUM - mfcc.shape[0]
        padding_zeros = np.zeros((padding_num, 39))
        
        padding_labels = np.zeros(padding_num)
        padding_labels = to_categorical(padding_labels, num_classes = 40)
        
        mfcc = np.concatenate((mfcc, padding_zeros), axis = 0)
        labels = np.concatenate((labels, padding_labels), axis = 0)
        


        
        X_data.append(mfcc)
        y_data.append(labels)
    X_data = np.array(X_data)
    y_data = np.array(y_data)
    print(X_data.shape)
    print(y_data.shape)
 
    return df
   

  


df = loadData()




        


['0', '1', '2', '4', '6', '7', '8', '37', '10', '11', '12', '13', '27', '29', '17', '18', '19', '20', '21', '22', '24', '25', '26', '28', '30', '31', '32', '33', '34', '36', '35', '39', '38', '40', '41', '42', '44', '45', '46']
(3696, 777, 39)
(3696, 777, 40)


In [39]:
df

,mfcc_0,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,...,mfcc_32,mfcc_33,mfcc_34,mfcc_35,mfcc_36,mfcc_37,mfcc_38,label,fid,f_name
faem0_si1392_1,38.35080,-31.048880,-10.463180,-9.166654,-11.218320,-1.998698,18.509480,3.447642,6.247809,-5.240960,...,-1.283352,0.217253,0.412052,0.622866,0.754879,1.063677,0.784817,7,1,faem0_si1392
faem0_si1392_2,38.35080,-29.885110,-2.339327,-2.430964,-3.292726,2.997352,5.760813,4.516098,14.786760,7.313382,...,0.100007,-0.231526,-1.060225,-1.233564,-0.346969,0.502436,0.100313,7,2,faem0_si1392
faem0_si1392_3,35.55493,-29.109250,-6.475107,-10.889740,0.982063,1.332002,-5.247100,0.954580,8.887121,10.708930,...,1.457594,-0.309879,-1.640108,-2.526095,-1.917202,-0.785079,-1.050887,7,3,faem0_si1392
faem0_si1392_4,34.15699,-34.540210,-9.724649,-12.299530,-6.915243,0.332792,4.418848,12.351440,22.254680,8.768617,...,1.101547,-1.125876,-2.326726,-2.163239,-1.330647,-0.808050,-1.391136,7,4,faem0_si1392
faem0_si1392_5,35.08895,-31.048880,-7.361346,-9.166654,-4.361423,-0.666418,-6.293220,1.666884,8.421360,4.887991,...,1.064093,-0.648734,-1.154530,-1.029233,-0.324148,-0.450126,-0.824935,7,5,faem0_si1392
faem0_si1392_6,34.15699,-31.824730,-8.542997,-7.756858,-7.609287,-6.039391,-6.293220,-2.607609,-2.446395,-2.375668,...,0.556591,-0.032228,0.847454,0.617231,0.948329,-0.335250,0.139213,7,6,faem0_si1392
faem0_si1392_7,35.55493,-37.643620,-3.373272,-7.756858,-4.717655,-0.333348,-0.278030,5.940705,4.540019,-7.352227,...,0.353947,0.427805,1.231791,1.305701,0.817838,-0.194944,1.196525,7,7,faem0_si1392
faem0_si1392_8,34.62297,-32.988510,-7.361346,-7.130281,-0.442867,-12.074320,-9.562345,-7.959957,-5.343700,0.037209,...,1.163020,1.336771,1.614035,1.000226,0.092871,-0.335167,1.074194,7,8,faem0_si1392
faem0_si1392_9,34.15699,-32.212660,-11.792540,-12.769460,-0.799099,-9.589352,-4.331745,-4.034903,-7.049210,-10.066710,...,0.368396,-0.390314,1.513048,0.580091,0.772962,-0.941735,0.144285,7,9,faem0_si1392
faem0_si1392_10,33.69101,-31.436810,-8.395291,-11.672960,-6.498817,-8.701862,-3.285625,-2.250786,7.179331,-3.129692,...,-0.155374,-2.146168,1.071831,-0.150805,1.183868,-1.853300,-0.336384,7,10,faem0_si1392


In [12]:
df


,mfcc_0,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,...,mfcc_32,mfcc_33,mfcc_34,mfcc_35,mfcc_36,mfcc_37,mfcc_38,label,fid,f_name
faem0_si1392_1,38.35080,-31.048880,-10.463180,-9.166654,-11.218320,-1.998698,18.509480,3.447642,6.247809,-5.240960,...,-1.283352,0.217253,0.412052,0.622866,0.754879,1.063677,0.784817,37,1,faem0_si1392
faem0_si1392_2,38.35080,-29.885110,-2.339327,-2.430964,-3.292726,2.997352,5.760813,4.516098,14.786760,7.313382,...,0.100007,-0.231526,-1.060225,-1.233564,-0.346969,0.502436,0.100313,37,2,faem0_si1392
faem0_si1392_3,35.55493,-29.109250,-6.475107,-10.889740,0.982063,1.332002,-5.247100,0.954580,8.887121,10.708930,...,1.457594,-0.309879,-1.640108,-2.526095,-1.917202,-0.785079,-1.050887,37,3,faem0_si1392
faem0_si1392_4,34.15699,-34.540210,-9.724649,-12.299530,-6.915243,0.332792,4.418848,12.351440,22.254680,8.768617,...,1.101547,-1.125876,-2.326726,-2.163239,-1.330647,-0.808050,-1.391136,37,4,faem0_si1392
faem0_si1392_5,35.08895,-31.048880,-7.361346,-9.166654,-4.361423,-0.666418,-6.293220,1.666884,8.421360,4.887991,...,1.064093,-0.648734,-1.154530,-1.029233,-0.324148,-0.450126,-0.824935,37,5,faem0_si1392
faem0_si1392_6,34.15699,-31.824730,-8.542997,-7.756858,-7.609287,-6.039391,-6.293220,-2.607609,-2.446395,-2.375668,...,0.556591,-0.032228,0.847454,0.617231,0.948329,-0.335250,0.139213,37,6,faem0_si1392
faem0_si1392_7,35.55493,-37.643620,-3.373272,-7.756858,-4.717655,-0.333348,-0.278030,5.940705,4.540019,-7.352227,...,0.353947,0.427805,1.231791,1.305701,0.817838,-0.194944,1.196525,37,7,faem0_si1392
faem0_si1392_8,34.62297,-32.988510,-7.361346,-7.130281,-0.442867,-12.074320,-9.562345,-7.959957,-5.343700,0.037209,...,1.163020,1.336771,1.614035,1.000226,0.092871,-0.335167,1.074194,37,8,faem0_si1392
faem0_si1392_9,34.15699,-32.212660,-11.792540,-12.769460,-0.799099,-9.589352,-4.331745,-4.034903,-7.049210,-10.066710,...,0.368396,-0.390314,1.513048,0.580091,0.772962,-0.941735,0.144285,37,9,faem0_si1392
faem0_si1392_10,33.69101,-31.436810,-8.395291,-11.672960,-6.498817,-8.701862,-3.285625,-2.250786,7.179331,-3.129692,...,-0.155374,-2.146168,1.071831,-0.150805,1.183868,-1.853300,-0.336384,37,10,faem0_si1392


In [24]:
mfcc_features = ['mfcc_' + str(i) for i in range(0,39)]
data = []
for rows in df:
    rows = rows[0].as_matrix(mfcc_features)
#         row = (row - np.mean(row, axis = 1)) / np.std(row, axis = 1)
    padding_num = MAX_SEQUENCE_NUM - rows.shape[0]
    padding_zeros = np.zeros((padding_num, 39))
    rows = np.concatenate((rows, padding_zeros), axis = 0)
   
        
  

    data.append(rows)
data = np.array(data)


print(data.shape)

(3696, 777, 39)


In [25]:
for idx, frame in enumerate(data):
    f_id = frame[0].split('_')
    f_name = f_id[0] + '_' + f_id[1]
    f_id = f_id[2]
    frame = np.append(frame, [f_id, f_name])
#     frame.append(f_id)
    print(frame)
    print(frame.shape)
    if idx > 3:
        break



['faem0_si1392_1' 38.3508 -31.04888 -10.46318 -9.166654 -11.21832
 -1.9986979999999999 18.50948 3.4476419999999997 6.247808999999999 -5.24096
 -4.613473 -14.820910000000001 -12.65494 -0.5591731 0.504303 1.61 0.3289518
 3.232636 1.165745 -6.0261830000000005 -0.39176669999999997 1.381758
 4.445412999999999 2.94172 2.511547 2.56285 -0.3261847 -0.16680770000000003
 -0.1314589 -0.4119736 0.2513791 -0.01998425 -1.283352 0.21725279999999997
 0.41205200000000003 0.6228664 0.7548792 1.063677 0.7848173 '37' '1'
 'faem0si1392']
(43,)
['faem0_si1392_10' 33.69101 -31.436809999999998 -8.395291 -11.67296
 -6.498817 -8.701862 -3.285625 -2.250786 7.1793309999999995 -3.129692
 -2.754778 -9.668961 -2.944329 0.04659748 0.7758503000000001 1.61 0.7362261
 0.890581 3.9399349999999997 3.404621 2.068165 0.7292350999999999 0.3920922
 0.45847829999999995 0.5244108000000001 2.290709 3.78355 1.9003290000000002
 -1.535199 0.5012606 -2.272216 -0.44922690000000004 -0.1553739 -2.146168
 1.071831 -0.150805 1.183868 -1.

In [ ]:
df['f_id'] = df.index
df['fid'] = df['f_id'].apply(lambda x: x.split('_')[2])
df['f_name'] =  df['f_id'].apply(lambda x: x.split('_')[0] + '_' + x.split('_')[1])
y = df
g = y.groupby('f_name')
y = np.array(list(g))
y = y
data = []
for d in y:
    data.append(d[1])
data


In [5]:
d = dict()
for index, row in df.iterrows():
    d[row['f_name']] = row